In [2]:
import requests
r=requests.get("http://e-service.cwb.gov.tw/HistoryDataQuery/MonthDataController.do?command=viewMain&station=466910&stname=%25E9%259E%258D%25E9%2583%25A8&datepicker=2019-08")
print(r.text)


<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html>
<head>
<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">
<script type="text/javascript">


function goErrorPage(){
	top.location.href = 'index.jsp';
}
</script>
</head>
<body onload="goErrorPage()">

</body>
</html>


In [3]:
import pandas as pd
fn_rain = "./WeatherStation_all.csv"
df_CurrentWeatherStation = pd.read_csv(fn_rain)
df_CurrentWeatherStation

,站號,站名,海拔高度(m),經度,緯度,城市,地址,資料起始日期,撤站日期,備註,原站號,新站號
0,466880,板橋,9.7,121.442017,24.997647,新北市,板橋區大觀路二段265巷62號,2002/1/1,NaN,原為探空站，自2002年開始進行氣象觀測。因受附近興建合宜住宅影響，日照觀測自民國104年起...,NaN,NaN
1,466900,淡水,19.0,121.448906,25.164889,新北市,淡水區中正東路42巷6號,1942/10/1,NaN,NaN,NaN,NaN
2,466910,鞍部,837.6,121.529731,25.182586,臺北市,北投區陽明山竹子湖路111號,1943/1/1,NaN,NaN,NaN,NaN
3,466920,臺北,5.3,121.514853,25.037658,臺北市,中正區公園路64號,1896/08/11,NaN,1992/02至1997/08因本局改建，觀測地點移至臺北師院(466921)。2014至2...,NaN,NaN
4,466930,竹子湖,607.1,121.544547,25.162078,臺北市,北投區陽明山竹子湖路2號,1943/1/1,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
674,C1T930,鳳林,130.0,121.445619,23.743603,花蓮縣,鳳林鎮鳳智里中正路2段1號(鳳林國民小學後方鐵道旁),1996/1/1,2017/5/4,因移位及升級，本站於2017/5/4起進行儀器汰換，於2017/6/3變更為(C0Z160)站。,NaN,C0Z160
675,C1T890,壽豐,40.0,121.507878,23.869097,花蓮縣,壽豐鄉壽豐村壽山路19號(壽豐國民小學操場旁),1996/1/1,2017/5/2,因移位，本站於2017/5/2起進行儀器汰換，於2017/6/3變更為(C1Z120)站。,NaN,C1Z120
676,C0C640,龜山,177.0,121.375719,25.030306,桃園市,龜山區舊路里振興路1169號(大埔國小校園內),2007/12/1,2017/3/22,NaN,NaN,NaN
677,C1V460,楠溪,1857.0,120.892611,23.436444,高雄市,桃源區(楠溪林道17k處愛玉工寮，車程約1小時),1993/3/6,2017/2/2,NaN,NaN,NaN


In [4]:
twStationList=list(df_CurrentWeatherStation['站號'])
len(twStationList)
altitudeList=list(df_CurrentWeatherStation['海拔高度(m)'])
len(altitudeList)

679

In [5]:
import requests
from bs4 import BeautifulSoup
import csv
twStationList = ["466910", "466920", "466930", "C0A980", "C0A990", "C0A9A0", "C0A9B0", "C0A9C0", "C0A9E0", "C0A9F0", "C0AC40", "C0AC70", "C0AC80", "C0AH40", "C0AH70", "C1A730", "C1AC50"]
monthSearch = [1,2,3,4, 5, 6, 7, 8, 9, 10, 11, 12]
yearList=['2019']
yearData=[]
s=""
def cdateList(year):
    # days31 (1,3,5,7,8,10,12) days30(2,4,6,9,11)
    month31=[1,3,5,7,8,10,12]
    yearData=[]
    s=""
    for month in monthSearch:
        if month in month31:
            for day in range(1, 32):
                s = year + '-' + str(month).zfill(2) + '-' + str(day).zfill(2)
                yearData.append(s)
        else :
            for day in range(1, 31):
                s = year + '-' + str(month).zfill(2) + '-' + str(day).zfill(2)
                yearData.append(s)
    return yearData

In [6]:
#執行爬蟲
import os
for stanum in range(len(twStationList)):
    for year in yearList:
        dateList = cdateList(year)
        for time in dateList:
            url="https://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station="+twStationList[stanum]+"&stname=%25E9%259E%258D%25E9%2583%25A8&datepicker="+time+"&altitude="+str(altitudeList[stanum])+"m"  #網址
            #csvfile="./weather_station/"+station+"/"+station+"-"+time+".csv" #開個csv檔案準備寫入

            #dir address 
            dirPath = "./weather_station_data/"+twStationList[stanum]

            if os.path.exists(dirPath) == 0:
                os.makedirs(dirPath)
            csvfile= dirPath+"/"+twStationList[stanum]+"-"+time+".csv"
            
            r=requests.get(url)
            r.encoding="utf-8-sig"
            soup=BeautifulSoup(r.text,"lxml")
            tag_table=soup.find(id="MyTable") #用BeautifulSoup找到table位置
            rows=tag_table.findAll("tr")
 
            with open(csvfile,"w+",newline="",encoding="utf-8") as fp:
                writer=csv.writer(fp)
                for row in rows[1:]:
                    rowList=[]
                    for cell in row.findAll(["td","th"]):
                        rowList.append(cell.get_text().replace("\n","").replace("\r",""))
                    writer.writerow(rowList)
            print(twStationList[stanum]+'-'+time)

466910-2019-01-01
466910-2019-01-02
466910-2019-01-03
466910-2019-01-04
466910-2019-01-05
466910-2019-01-06
466910-2019-01-07
466910-2019-01-08
466910-2019-01-09
466910-2019-01-10
466910-2019-01-11
466910-2019-01-12
466910-2019-01-13
466910-2019-01-14
466910-2019-01-15
466910-2019-01-16
466910-2019-01-17
466910-2019-01-18
466910-2019-01-19
466910-2019-01-20
466910-2019-01-21
466910-2019-01-22
466910-2019-01-23
466910-2019-01-24
466910-2019-01-25
466910-2019-01-26
466910-2019-01-27
466910-2019-01-28
466910-2019-01-29
466910-2019-01-30
466910-2019-01-31
466910-2019-02-01
466910-2019-02-02
466910-2019-02-03
466910-2019-02-04
466910-2019-02-05
466910-2019-02-06
466910-2019-02-07
466910-2019-02-08
466910-2019-02-09
466910-2019-02-10
466910-2019-02-11
466910-2019-02-12
466910-2019-02-13
466910-2019-02-14
466910-2019-02-15
466910-2019-02-16
466910-2019-02-17
466910-2019-02-18
466910-2019-02-19
466910-2019-02-20
466910-2019-02-21
466910-2019-02-22
466910-2019-02-23
466910-2019-02-24
466910-201